# Operational Intensity Analysis Tool

In [12]:
import numpy as np
from keras.applications import VGG16, MobileNet, Xception, ResNet50, InceptionV3

In [2]:
def get_kernel_size(layer):
    layer_type = layer.__class__.__name__
    if 'Conv2D' in layer_type or 'Dense' in layer_type:
        weights = layer.get_weights()
        return weights[0].shape
    else:
        return 0

In [3]:
def getFLOPS(layer, kernel, output):
    # For Xception
    if "Separable" in layer:
        return np.prod(output) * (np.prod(kernel[:-2]) + kernel[-2])
    # For MobileNet
    elif "Depthwise" in layer:
        return np.prod(output) * np.prod(kernel[:-2])
    # Regular Convolution
    elif "Conv2D" in layer or "Dense" in layer:
        return np.prod(output) * np.prod(kernel[:-1])
    else:
        return 0

In [11]:
def intensity(model):
    print('Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS')
    print('--- | --- | --- | --- | --- | --- | ---')
    sum_kernel_mem = 0
    sum_output_mem = 0
    sum_flops = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        kernel_size = get_kernel_size(l)
        kernel_mem = np.prod(kernel_size)
        output_size = l.output_shape[1:]
        output_mem = np.prod(output_size)
        flops = getFLOPS(layer_type, kernel_size, output_size)
        print(l.name, '|', layer_type, '|', 
              kernel_size, '|', "{:,}".format(kernel_mem), '|', 
              output_size, '|', "{:,}".format(output_mem), '|', 
              "{:,}".format(flops))
        sum_kernel_mem += kernel_mem
        sum_output_mem += output_mem
        sum_flops += flops
    print('- | - | - | - | - | - | -')
    print('- | - | - | Kernel Mem (Total) | - | Output Mem (Total) | FLOPS (Total)')
    print('Summary | - | - |', 
          "{:,}".format(sum_kernel_mem), '| - |', 
          "{:,}".format(sum_output_mem), '|', 
          "{:,}".format(sum_flops))
    print('\n----------------------\n')
    print('Model Name: %s' % model.name)
    print('Overall FLOPS: %.f MFLOPS' % (sum_flops/1000/1000))
    print('Overall Params: %.f MParams' % ((sum_kernel_mem + sum_output_mem)/1024/1024))
    # Default dtype is float32 (4Byte)
    op_intensity = sum_flops / ((sum_kernel_mem + sum_output_mem) * 4)
    print('Operational Intensity = %.f FLOPS/Byte' % op_intensity)

In [6]:
"""MobileNet v2 models for Keras.

# Reference
- [Inverted Residuals and Linear Bottlenecks Mobile Networks for
   Classification, Detection and Segmentation]
   (https://arxiv.org/abs/1801.04381)
"""


from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, add, Reshape
from keras.applications.mobilenet import relu6, DepthwiseConv2D
from keras.utils.vis_utils import plot_model

from keras import backend as K


def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.

    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.

    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    return Activation(relu6)(x)


def _bottleneck(inputs, filters, kernel, t, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.

    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        r: Boolean, Whether to use the residuals.

    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation(relu6)(x)

    x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = add([x, inputs])
    return x


def _inverted_residual_block(inputs, filters, kernel, t, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.

    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """

    x = _bottleneck(inputs, filters, kernel, t, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, 1, True)

    return x


def MobileNetv2(input_shape, k):
    """MobileNetv2
    This function defines a MobileNetv2 architectures.

    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
    # Returns
        MobileNetv2 model.
    """

    inputs = Input(shape=input_shape)
    x = _conv_block(inputs, 32, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, strides=1, n=1)

    x = _conv_block(x, 1280, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, 1280))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)
    plot_model(model, to_file='MobileNetv2.png', show_shapes=True)

    return model

In [18]:
intensity(MobileNetv2((224, 224, 3), 1000))

Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS
--- | --- | --- | --- | --- | --- | ---
input_10 | InputLayer | 0 | 0 | (224, 224, 3) | 150,528 | 0
conv2d_300 | Conv2D | (3, 3, 3, 32) | 864 | (112, 112, 32) | 401,408 | 10,838,016
batch_normalization_348 | BatchNormalization | 0 | 0 | (112, 112, 32) | 401,408 | 0
activation_346 | Activation | 0 | 0 | (112, 112, 32) | 401,408 | 0
conv2d_301 | Conv2D | (1, 1, 32, 32) | 1,024 | (112, 112, 32) | 401,408 | 12,845,056
batch_normalization_349 | BatchNormalization | 0 | 0 | (112, 112, 32) | 401,408 | 0
activation_347 | Activation | 0 | 0 | (112, 112, 32) | 401,408 | 0
depthwise_conv2d_52 | DepthwiseConv2D | (3, 3, 32, 1) | 288 | (112, 112, 32) | 401,408 | 3,612,672
batch_normalization_350 | BatchNormalization | 0 | 0 | (112, 112, 32) | 401,408 | 0
activation_348 | Activation | 0 | 0 | (112, 112, 32) | 401,408 | 0
conv2d_302 | Conv2D | (1, 1, 32, 16) | 512 | (112, 112, 16) | 200,704 | 6,422,528
batch_normali

In [17]:
intensity(VGG16())

Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS
--- | --- | --- | --- | --- | --- | ---
input_9 | InputLayer | 0 | 0 | (224, 224, 3) | 150,528 | 0
block1_conv1 | Conv2D | (3, 3, 3, 64) | 1,728 | (224, 224, 64) | 3,211,264 | 86,704,128
block1_conv2 | Conv2D | (3, 3, 64, 64) | 36,864 | (224, 224, 64) | 3,211,264 | 1,849,688,064
block1_pool | MaxPooling2D | 0 | 0 | (112, 112, 64) | 802,816 | 0
block2_conv1 | Conv2D | (3, 3, 64, 128) | 73,728 | (112, 112, 128) | 1,605,632 | 924,844,032
block2_conv2 | Conv2D | (3, 3, 128, 128) | 147,456 | (112, 112, 128) | 1,605,632 | 1,849,688,064
block2_pool | MaxPooling2D | 0 | 0 | (56, 56, 128) | 401,408 | 0
block3_conv1 | Conv2D | (3, 3, 128, 256) | 294,912 | (56, 56, 256) | 802,816 | 924,844,032
block3_conv2 | Conv2D | (3, 3, 256, 256) | 589,824 | (56, 56, 256) | 802,816 | 1,849,688,064
block3_conv3 | Conv2D | (3, 3, 256, 256) | 589,824 | (56, 56, 256) | 802,816 | 1,849,688,064
block3_pool | MaxPooling2D | 0 | 0 |

In [16]:
intensity(MobileNet())

Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS
--- | --- | --- | --- | --- | --- | ---
input_8 | InputLayer | 0 | 0 | (224, 224, 3) | 150,528 | 0
conv1 | Conv2D | (3, 3, 3, 32) | 864 | (112, 112, 32) | 401,408 | 10,838,016
conv1_bn | BatchNormalization | 0 | 0 | (112, 112, 32) | 401,408 | 0
conv1_relu | Activation | 0 | 0 | (112, 112, 32) | 401,408 | 0
conv_dw_1 | DepthwiseConv2D | (3, 3, 32, 1) | 288 | (112, 112, 32) | 401,408 | 3,612,672
conv_dw_1_bn | BatchNormalization | 0 | 0 | (112, 112, 32) | 401,408 | 0
conv_dw_1_relu | Activation | 0 | 0 | (112, 112, 32) | 401,408 | 0
conv_pw_1 | Conv2D | (1, 1, 32, 64) | 2,048 | (112, 112, 64) | 802,816 | 25,690,112
conv_pw_1_bn | BatchNormalization | 0 | 0 | (112, 112, 64) | 802,816 | 0
conv_pw_1_relu | Activation | 0 | 0 | (112, 112, 64) | 802,816 | 0
conv_dw_2 | DepthwiseConv2D | (3, 3, 64, 1) | 576 | (56, 56, 64) | 200,704 | 1,806,336
conv_dw_2_bn | BatchNormalization | 0 | 0 | (56, 56, 64) | 200,704

In [9]:
intensity(Xception(input_shape=(299, 299, 3)))

Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS
--- | --- | --- | --- | --- | --- | ---
input_4 | InputLayer | 0 | 0 | (299, 299, 3) | 268,203 | 0
block1_conv1 | Conv2D | (3, 3, 3, 32) | 864 | (149, 149, 32) | 710,432 | 19,181,664
block1_conv1_bn | BatchNormalization | 0 | 0 | (149, 149, 32) | 710,432 | 0
block1_conv1_act | Activation | 0 | 0 | (149, 149, 32) | 710,432 | 0
block1_conv2 | Conv2D | (3, 3, 32, 64) | 18,432 | (147, 147, 64) | 1,382,976 | 398,297,088
block1_conv2_bn | BatchNormalization | 0 | 0 | (147, 147, 64) | 1,382,976 | 0
block1_conv2_act | Activation | 0 | 0 | (147, 147, 64) | 1,382,976 | 0
block2_sepconv1 | SeparableConv2D | (3, 3, 64, 1) | 576 | (147, 147, 128) | 2,765,952 | 201,914,496
block2_sepconv1_bn | BatchNormalization | 0 | 0 | (147, 147, 128) | 2,765,952 | 0
block2_sepconv2_act | Activation | 0 | 0 | (147, 147, 128) | 2,765,952 | 0
block2_sepconv2 | SeparableConv2D | (3, 3, 128, 1) | 1,152 | (147, 147, 128) | 2,765,952 

In [15]:
intensity(ResNet50())

Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS
--- | --- | --- | --- | --- | --- | ---
input_7 | InputLayer | 0 | 0 | (224, 224, 3) | 150,528 | 0
conv1 | Conv2D | (7, 7, 3, 64) | 9,408 | (112, 112, 64) | 802,816 | 118,013,952
bn_conv1 | BatchNormalization | 0 | 0 | (112, 112, 64) | 802,816 | 0
activation_297 | Activation | 0 | 0 | (112, 112, 64) | 802,816 | 0
max_pooling2d_9 | MaxPooling2D | 0 | 0 | (55, 55, 64) | 193,600 | 0
res2a_branch2a | Conv2D | (1, 1, 64, 64) | 4,096 | (55, 55, 64) | 193,600 | 12,390,400
bn2a_branch2a | BatchNormalization | 0 | 0 | (55, 55, 64) | 193,600 | 0
activation_298 | Activation | 0 | 0 | (55, 55, 64) | 193,600 | 0
res2a_branch2b | Conv2D | (3, 3, 64, 64) | 36,864 | (55, 55, 64) | 193,600 | 111,513,600
bn2a_branch2b | BatchNormalization | 0 | 0 | (55, 55, 64) | 193,600 | 0
activation_299 | Activation | 0 | 0 | (55, 55, 64) | 193,600 | 0
res2a_branch2c | Conv2D | (1, 1, 64, 256) | 16,384 | (55, 55, 256) | 774,400 | 49

In [14]:
intensity(InceptionV3(input_shape=(299, 299, 3)))

Layer Name | Layer Type | Kernel Size | Kernel Mem | Output Size | Output Mem | FLOPS
--- | --- | --- | --- | --- | --- | ---
input_6 | InputLayer | 0 | 0 | (299, 299, 3) | 268,203 | 0
conv2d_206 | Conv2D | (3, 3, 3, 32) | 864 | (149, 149, 32) | 710,432 | 19,181,664
batch_normalization_254 | BatchNormalization | 0 | 0 | (149, 149, 32) | 710,432 | 0
activation_203 | Activation | 0 | 0 | (149, 149, 32) | 710,432 | 0
conv2d_207 | Conv2D | (3, 3, 32, 32) | 9,216 | (147, 147, 32) | 691,488 | 199,148,544
batch_normalization_255 | BatchNormalization | 0 | 0 | (147, 147, 32) | 691,488 | 0
activation_204 | Activation | 0 | 0 | (147, 147, 32) | 691,488 | 0
conv2d_208 | Conv2D | (3, 3, 32, 64) | 18,432 | (147, 147, 64) | 1,382,976 | 398,297,088
batch_normalization_256 | BatchNormalization | 0 | 0 | (147, 147, 64) | 1,382,976 | 0
activation_205 | Activation | 0 | 0 | (147, 147, 64) | 1,382,976 | 0
max_pooling2d_5 | MaxPooling2D | 0 | 0 | (73, 73, 64) | 341,056 | 0
conv2d_209 | Conv2D | (1, 1, 64, 